# 第6章: 機械学習
本章では，Fabio Gasparetti氏が公開しているNews Aggregator Data Setを用い，ニュース記事の見出しを「ビジネス」「科学技術」「エンターテイメント」「健康」のカテゴリに分類するタスク（カテゴリ分類）に取り組む．

## 50. データの入手・整形
News Aggregator Data Setをダウンロードし、以下の要領で学習データ（train.txt），検証データ（valid.txt），評価データ（test.txt）を作成せよ．

1.ダウンロードしたzipファイルを解凍し，readme.txtの説明を読む．


2.情報源（publisher）が”Reuters”, “Huffington Post”, “Businessweek”, “Contactmusic.com”, “Daily Mail”の事例（記事）のみを抽出する．


3.抽出された事例をランダムに並び替える．


4.抽出された事例の80%を学習データ，残りの10%ずつを検証データと評価データに分割し，それぞれtrain.txt，valid.txt，test.txtというファイル名で保存する．ファイルには，１行に１事例を書き出すこととし，カテゴリ名と記事見出しのタブ区切り形式とせよ（このファイルは後に問題70で再利用する）．


5.学習データと評価データを作成したら，各カテゴリの事例数を確認せよ

参考情報
- データの削除：https://note.nkmk.me/python-pandas-query/
- シャッフル：https://note.nkmk.me/python-pandas-random-sort-shuffle/

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [24]:
def data_split(data):
    df_news = pd.read_csv(data,header=None,sep='\t',names=['ID','TITLE','URL','PUBLISHER','CATEGORY',
                                                                                               'STORY','HOSTNAME','TIMESTAMP'])
    #必要な記事を抽出してランダムに入れ替える
    df_news=df_news[df_news['PUBLISHER'].isin(["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"])]
    df_news_suffle = df_news.sample(frac=1,random_state=0)
    
    #データの分割
    train,temp = train_test_split(df_news_suffle,test_size=0.2 )
    val,test=train_test_split(temp,test_size=0.5)
    
    #データの保存
    train.to_csv('../data/train.txt', sep='\t', index=False)
    val.to_csv('../data/valid.txt', sep='\t', index=False)
    test.to_csv('../data/test.txt', sep='\t', index=False)
    
    #学習データと評価データの事例の確認
    ans1=print("訓練データ\n{}".format(train['CATEGORY'].value_counts()))
    ans2=print("検証データ\n{}".format(val['CATEGORY'].value_counts()))
    ans3=print("評価データ\n{}".format(test['CATEGORY'].value_counts()))
    
    return ans1,ans2
    

In [5]:
data = '../data/chap6/NewsAggregatorDataset/newsCorpora.csv'

In [6]:
data_split(data)

訓練データ
b    4521
e    4213
t    1221
m     717
Name: CATEGORY, dtype: int64
検証データ
b    551
e    523
t    155
m    105
Name: CATEGORY, dtype: int64
評価データ
b    555
e    543
t    148
m     88
Name: CATEGORY, dtype: int64


(None, None)

## 51. 特徴量抽出Permalink
学習データ，検証データ，評価データから特徴量を抽出し，それぞれtrain.feature.txt，valid.feature.txt，test.feature.txtというファイル名で保存せよ． なお，カテゴリ分類に有用そうな特徴量は各自で自由に設計せよ．記事の見出しを単語列に変換したものが最低限のベースラインとなるであろう

ニュース記事をロジ回帰するにはTF-IDFがいいらしい：https://poniti.hatenablog.com/entry/2020/01/02/185114

一般的に自然言語の特徴抽出に使われるのは以下
<br>BOW
<br>TF-IDF(単語の頻度だけでなく、重要度も考慮)
<br>BM25（TF-IDFより文章の長さを考慮)
<br>N-gram


In [19]:
#今回はTITLEをカテゴリーに分類するのが目的

In [37]:
train_data = '../data/chap6/train.txt'
train = pd.read_table(train_data)
train.head()

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
0,107117,Paul Stanley Claims Some KISS Bandmates Were A...,http://www.huffingtonpost.com/2014/04/07/kiss-...,Huffington Post,e,ds2zAAbxdE87cfMiYsLN_YN0_inyM,www.huffingtonpost.com,1397355178491
1,318767,UPDATE 4-GM prepares to recall some Cruze seda...,http://in.reuters.com/article/2014/06/26/gm-cr...,Reuters,t,duJ0gOzwA-dJd9MNppj3e4KyFBznM,in.reuters.com,1403787357244
2,406824,Tesla Breaks Ground as Nevada Takes Lead for N...,http://www.businessweek.com/news/2014-07-31/te...,Businessweek,t,d_6CT2UjnUp12gMtzr7_n0kjjW_VM,www.businessweek.com,1406860215424
3,146904,Ackman's Pershing Amassed Allergan Stake Under...,http://www.businessweek.com/news/2014-04-21/ac...,Businessweek,b,dfWBqDxgv21TUlMFBrhOOHpO3cvPM,www.businessweek.com,1398160802731
4,304247,Newlywed Kim Kardashian shares trailer of her ...,http://www.dailymail.co.uk/tvshowbiz/article-2...,Daily Mail,e,d6YxcE79pJ_0tiMRWbOCbd8q2yX_M,www.dailymail.co.uk,1403137679773


In [39]:
print(train['TITLE'])

0        Paul Stanley Claims Some KISS Bandmates Were A...
1        UPDATE 4-GM prepares to recall some Cruze seda...
2        Tesla Breaks Ground as Nevada Takes Lead for N...
3        Ackman's Pershing Amassed Allergan Stake Under...
4        Newlywed Kim Kardashian shares trailer of her ...
                               ...                        
10667    Julia Louis-Dreyfus's show Veep renewed for fo...
10668       Don't Let Gluten-Free Become the Next Fat-Free
10669    Transformers: Age of Extinction reviewed. | Pa...
10670    REFILE-Reynolds American profit rises 6.7 pct ...
10671     Could Penguin Beat Amazon at Book Subscriptions?
Name: TITLE, Length: 10672, dtype: object


In [41]:
train_title=train['TITLE']


In [23]:
val.count()

ID           1334
TITLE        1334
URL          1334
PUBLISHER    1334
CATEGORY     1334
STORY        1334
HOSTNAME     1334
TIMESTAMP    1334
dtype: int64